In [1]:
#https://stackabuse.com/accessing-the-twitter-api-with-python/
# to save my twitter_credentials.json
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY'] = 'lrPS9OHXHvE035nktii81w5BF'
credentials['CONSUMER_SECRET'] = 'pF76sVbMXkVPAZZ4eqaDLsQBMeFKCYctvJoPxlamqje8WHq8n7' 
credentials['ACCESS_TOKEN'] = '757203324970102784-3El1NyAOlq0oulCAzuDfj92Jp0L0fiy'
credentials['ACCESS_SECRET'] = 'EEpo2ZfpRINSDoh5amp8Jw8Zj985KVRutIwSLKTrOt2lZ'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)

In [3]:
# Import the Twython class
from twython import Twython  
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
#We'll use only four arguments in the query: 
#q, result_type, count and lang, respectively for the search keyword, type, count, and language of results. 
query = {'q': 'learn python',  
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }

In [4]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}  
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  
df.head(5)  

,user,date,text,favorite_count
2,KadariaAhmed,Fri Mar 22 10:57:37 +0000 2019,Want to continue to be relevant in the world? ...,553
3,DataCamp,Fri Mar 22 18:00:21 +0000 2019,New Course by @_inesmontani: Advanced NLP with...,113
1,DataCamp,Mon Mar 25 01:00:11 +0000 2019,"In this Python Excel tutorial, learn how to re...",41
0,DataCamp,Mon Mar 25 12:00:50 +0000 2019,Keras Tutorial: Deep Learning in Python! This ...,37


In [8]:
from twython import TwythonStreamer  
import csv

# Filter out unwanted data
def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [ ]:
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='python')  

In [ ]:
import pandas as pd  
tweets = pd.read_csv("saved_tweets.csv")  
tweets.head()  

In [ ]:
from collections import Counter  
import ast

tweets = pd.read_csv("saved_tweets.csv")

# Extract hashtags and put them in a list
list_hashtag_strings = [entry for entry in tweets.hashtags]  
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))  
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)  
counter_hashtags.most_common(20)  

In [7]:
from geopy.geocoders import Nominatim  
import gmplot

geolocator = Nominatim()

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}  
for count, user_loc in enumerate(tweets.location):  
    try:
        location = geolocator.geocode(user_loc)

        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)

    # If too many connection requests
    except:
        pass

# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# Save the map to html file
gmap.draw("python_heatmap.html")  

ModuleNotFoundError: No module named 'geopy'